<a href="https://colab.research.google.com/github/NLE2-Arbeitsgruppe/QUASCoT/blob/main/TEST_0410.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# csv -> json: für Covid-Q => ÜBERSPRINGEN!!!!!!!!!!!!
# Einlesen von Covid-Q
import csv
import json
 
 
# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # Assuming a column named 'No' to
            # be the primary key
            key = rows['Category']
            data[key] = rows
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
         
# Driver Code
 
# Decide the two file paths according to your
# computer system
csvFilePath = r'final_master_dataset.csv'
jsonFilePath = r'final_master_dataset.json'
 
# Call the make_json function
make_json(csvFilePath, jsonFilePath)

FileNotFoundError: ignored

# QUASCoT - a question answering system for corona topics

Please enable the  GPU runtime (Runtime -> Change Runtime type -> Hardware accelerator -> GPU). To check whether the GPU is enabled run the following command:

In [ ]:
%%bash

nvidia-smi

Wed Oct  5 15:20:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-ovuhkf9f/farm-haystack_783bc42b848b4e59b21098c86bfeb219
  Resolved https://github.com/deepset-ai/haystack.git to commit 6cb4e939653719698764b2e0ad379b0859f862be
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━

  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-ovuhkf9f/farm-haystack_783bc42b848b4e59b21098c86bfeb219


In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

## ElasticsearchDocumentStore

We chose the `ElasticsearchDocumentStore` cause of the multiple advantages related to the provided and preloaded features. In addition, the use of `ElasticsearchDocumentStore` is recommended by Haystack. Because we ran the code in `Google Colab` it is not possible to start ELasticsearch (ES) using Docker.

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time
time.sleep(30)

In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="quascot_data")
#document_store = ElasticsearchDocumentStore(host=host, username="elastic", password="7V1zoL4SxbQnMJ8TKazc", index="document")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


You can skip the following commented out code block. By loading a JSON file we already have data in dict format  as the command `type(json_files)`confirms. 

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http


# Let's first fetch some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
# doc_dir = "data/tutorial1"
# s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip"
# fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
# all_coda = convert_files_to_docs(dir_path="/content/final_master_dataset.json", clean_func= clean_wiki_text, split_paragraphs=True)

# dicts = 
# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is:
# {
#    'content': "<DOCUMENT_TEXT_HERE>",
#    'meta': {'name': "<DOCUMENT_NAME_HERE>", ...}
# }
# (Optionally: you can also add more key-value-pairs here, that will be indexed as fields in Elasticsearch and
# can be accessed later for filtering or shown in the responses of the Pipeline)

# Let's have a look at the first 3 entries:

# Now, let's write the dicts containing documents to our DB.
# document_store.write_documents(all_coda)

Before you run the next code block make sure that you uploaded the `COVID-QA.json` file in the sample_data folder. You can find this folder by clicking on the folder icon on the upper left menu. Click on the 3 dot symbol and choose "upload" to upload the .json file. Wait until you can see the `COVID-QA.json` file in the folder overview.

In [ ]:
#with open('final_master_dataset.json') as json_file:
 #   data = json.loads(open((json_file))

# Datei laden
import json
json_files = json.loads(open('/content/sample_data/COVID-QA.json').read())

In [ ]:
[]

[]

In [ ]:
type(json_files)

dict

In [ ]:
json_files

In [ ]:
import pandas as pd
from haystack.utils.preprocessing import convert_files_to_docs

#df = pd.read_json(r"/content/final_master_dataset.json")
#convert_files_to_docs(dir_path=r"/content/final_master_dataset.json")

### Excursion

If you are unsure whether indexing data in elasticsearch works in general, you can test this by running the following code. Please note that this will overwrite the original`json_files` content. So you will have to re-run the code to read the data set.

In [ ]:
# Test, ob ES funktioniert/Indexerstellung

json_files = [{
    'content' : 'text',
    'meta': {
		'paragraphs': [{
			'qas': [{
					'question': 'text',
					'id': 'int',
					'answers': 'text' [{
						'text': 'text',
						'answer_start': 'int',
					}],
					'is_impossible': 'bool'
				},
			]
}]}
}]
               
    

document_store.write_documents(json_files)


In [ ]:
type(json_files)

list

In [ ]:
# Test funktioniert
print(document_store)
document_store.get_all_documents()

[<Document: {'content': 'text', 'content_type': 'text', 'score': None, 'meta': {}, 'embedding': None, 'id': '9cecc1d309197d54c2679c7f49f4e278'}>]

### Excursus end

In [ ]:
from elasticsearch import helpers, Elasticsearch
import pandas as pd
import json

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
index_name = 'quascot_data'
doctype = '_doc'
#es.indices.delete(index=index_name, ignore=[400, 404])
es.indices.create(index=index_name, ignore=400)
action_list = []
for paragraphs in json_files:
    record ={
        '_op_type': 'index',
        '_index': index_name,
        '_type' : doctype,
        '_source': paragraphs
    }
    action_list.append(record)
helpers.bulk(es, action_list)

## Preprocessor

In [ ]:
# Preprocessing des datasets
import nltk

from haystack.nodes import PreProcessor

preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=200, #200, da …
    split_respect_sentence_boundary=True,
)

docs = preprocessor.process(json_files)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

## FARMReader


In [ ]:
from haystack.nodes import BM25Retriever

#Retriever anpassen, kein Hindernis durch Elasticsearch, da ES mit allen Retrievern kompatibel ist
# BM25 is a variant of TF-IDF -> sparse vectors vs. Embedding Retrieval: vgl. CI: für jedes Wort Beziehungen, große Datenmenge zu Training, wurde bei Übersetzungsstudienleistung verwendet 
retriever = BM25Retriever(document_store=document_store)

In [ ]:
from haystack.nodes import FARMReader

# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

# laut Paper zu Covid-QA wurde eine RoBERTa-base Architektur ausgewählt und mit SQUaD finetuning betrieben
reader = FARMReader(model_name_or_path="armageddon/roberta-large-squad2-covid-qa-deepset", use_gpu=True)

# reader trainieren mit der reader.train() Funktion!!!!!
# BioBert: vgl. Huggingface Suchanfrage: BioBert, Filter: Question Answering, Dataset = deepset/covid-QA
# SciBERT: "allenai/scibert_scivocab_uncased" (https://huggingface.co/allenai/scibert_scivocab_uncased)!!!!!

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'armageddon/roberta-large-squad2-covid-qa-deepset' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'armageddon/roberta-large-squad2-covid-qa-deepset' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
prediction = pipe.run(
    query="How many nucleotides does bovine coronavirus contain?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

In [ ]:
from haystack.utils import print_answers

# Change `minimum` to `medium` or `all` to raise the level of detail
print_answers(prediction, details="all")


Query: How many nucleotides does bovine coronavirus contain?
Answers:
[]


In [ ]:
from pprint import pprint

pprint(prediction)

{'answers': [],
 'documents': [],
 'node_id': 'Reader',
 'params': {'Reader': {'top_k': 5}, 'Retriever': {'top_k': 10}},
 'query': 'How many nucleotides does bovine coronavirus contain?',
 'root_node': 'Query'}


In [ ]:
# !!!! Daten so niedrig, weil mit überschriebenen json_files weiter ausgeführt
import requests
print(requests.get("http://localhost:9200/_cat/indices").text)

yellow open document fw4ymQh0Q9WPo-5CLfEsHg 1 1 1 0 5.4kb 5.4kb
yellow open label    r8rCsLTyRTyofa5xcNK5rw 1 1 0 0  208b  208b



## TransformerReader

show the exact difference in effectiveness between FARM and TransformerReader 